# osiris

![img](https://dm2301files.storage.live.com/y4mmRC1xelS6Y6MEqUnZ-k2vjpADHpo6UMZAaZWROunr9-Ml5FYDlZ6WMxCGedy7NDhwDpusZdF5E1oLR5Qn6momydHe7tYUOMwNeFeGW7pUWkBjGPSnZp2sacYWs9IKkose6xjhSySL_v2tbfItRI7T_Pw_Tayhaa2F_vrwW6ucyr6WPa6s9DWH_if9Y5Y3yAU?width=375&height=250&cropmode=none)


osiris is a Python data processing and analysis environment for data-based computational conflict forecasting using very large datasets and graph-based queries and methods and models and visualization powered by scalable graph databases.

This notebook gives an overview of the osiris project. It describes the [GDELT project](https://www.gdeltproject.org/) data that osiris uses, how to import it using osiris either from the GDELT file server or from Google BigQuery, and how visualize and analyze it using Python, how to load it into a TigerGraph graph server instance, and then run queries on it..

In [111]:
# Check if running inside Colab
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

In [ ]:
# Uncomment and run below if running inside Colab and you want to pull env variables from a file called vars.env on your GDrive
# !pip install colab-env --upgrade
# import colab_env

In [ ]:
# Uncomment and run below if you need to install osiris e.g. if running inside Colab
# !if [ -d "osiris" ]; then rm -Rf osiris; fi
# !git clone https://github.com/allisterb/osiris --recurse-submodules
# !cd osiris && ./install

In [1]:
# Import the osiris code and set the runtime env. 
import sys
if IN_COLAB:
    sys.path.append(os.path.join('osiris', 'osiris'))
    sys.path.append(os.path.join('osiris', 'ext'))
else: # assuming we're running from the osiris project folder
    sys.path.append(os.path.join('..', 'osiris'))
    sys.path.append(os.path.join('..', 'ext'))
from osiris_global import set_runtime_env
set_runtime_env(interactive_nb=True)

# GDELT Event Data

*From the  [GDELT project](https://www.gdeltproject.org/) website*:
>The GDELT Project is a realtime network diagram and database of global human society for open research.
![gf](https://www.gdeltproject.org/images/spinningglobe.gif)
>The GDELT Project is an initiative to construct a catalog of human societal-scale behavior and beliefs across all countries of the world, connecting every person, organization, location, count, theme, news source, and event across the planet into a single massive network that captures what's happening around the world, what its context is and who's involved, and how the world is feeling about it, every single day.

The GDELT [event data](http://data.gdeltproject.org/documentation/GDELT-Event_Codebook-V2.0.pdf) contains hundreds of millions of automatically coded events extracted from news stories daily. osiris allows you to extract data directly from the GDELT file server/ The advantage of this method is that you don't need to have any special credentials or server access (remember we want *open-source* indicators.). All the data is downloaded directly to your client machine.

In [2]:
# Import data directly from GDELT file server
from data.gdelt import DataSource
import pandas as pd
gdelt = DataSource()

In [3]:
# Get event data for a 1 week period
events = gdelt.import_data('events', 'Apr-14-2022', 'Apr-20-2022')

Importing GDELT events data for 7 day(s) from 04-14-2022 to 04-20-2022...


Import GDELT events data:   0%|          | 0/7 [00:00<?, ?day/s]

Importing GDELT events data for 7 day(s) from 04-14-2022 to 04-20-2022 completed in 70.02 s.


About a week's worth of event data in 2022 consists of about 700K events takes up about 340MB RAM.

In [4]:
events.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 707186 entries, 0 to 125669
Data columns (total 62 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   GLOBALEVENTID          707186 non-null  int64  
 1   SQLDATE                707186 non-null  int64  
 2   MonthYear              707186 non-null  int64  
 3   Year                   707186 non-null  int64  
 4   FractionDate           707186 non-null  float64
 5   Actor1Code             640700 non-null  object 
 6   Actor1Name             640700 non-null  object 
 7   Actor1CountryCode      408112 non-null  object 
 8   Actor1KnownGroupCode   9610 non-null    object 
 9   Actor1EthnicCode       3423 non-null    object 
 10  Actor1Religion1Code    10452 non-null   object 
 11  Actor1Religion2Code    2561 non-null    object 
 12  Actor1Type1Code        296023 non-null  object 
 13  Actor1Type2Code        19713 non-null   object 
 14  Actor1Type3Code        495 non-null 

In [5]:
events

,GLOBALEVENTID,SQLDATE,MonthYear,Year,FractionDate,Actor1Code,Actor1Name,Actor1CountryCode,Actor1KnownGroupCode,Actor1EthnicCode,...,ActionGeo_Type,ActionGeo_FullName,ActionGeo_CountryCode,ActionGeo_ADM1Code,ActionGeo_ADM2Code,ActionGeo_Lat,ActionGeo_Long,ActionGeo_FeatureID,DATEADDED,SOURCEURL
0,1039303078,20210414,202104,2021,2021.2849,CAN,CANADA,CAN,NaN,NaN,...,4,"Port Elgin, Ontario, Canada",CA,CA08,12643,44.4333,-81.38330,-571576,20220414014500,https://www.lakeshoreadvance.com/news/local-ne...
1,1039303079,20210414,202104,2021,2021.2849,CAN,CANADA,CAN,NaN,NaN,...,4,"Port Elgin, Ontario, Canada",CA,CA08,12643,44.4333,-81.38330,-571576,20220414014500,https://www.lakeshoreadvance.com/news/local-ne...
2,1039303080,20210414,202104,2021,2021.2849,CHN,CHINA,CHN,NaN,NaN,...,4,"Shanghai, Shanghai, China",CH,CH23,13243,31.2222,121.45800,-1924465,20220414014500,https://news.yahoo.com/zealand-court-rules-all...
3,1039303081,20210414,202104,2021,2021.2849,CVL,SCIENTIST,NaN,NaN,NaN,...,4,"Paris, France (general), France",FR,FR00,16282,48.8667,2.33333,-1456928,20220414014500,http://www.jordantimes.com/news/features/first...
4,1039303082,20210414,202104,2021,2021.2849,MNCUSAMED,GOOGLE,USA,NaN,NaN,...,2,"California, United States",US,USCA,NaN,36.1700,-119.74600,CA,20220414014500,https://menafn.com/1104016162/Google-to-invest...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125665,1040383216,20220420,202204,2022,2022.3014,cre,CREE,NaN,NaN,cre,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20220420234500,https://www.cjvr.com/2022/04/20/first-nations-...
125666,1040383217,20220420,202204,2022,2022.3014,cre,CREE,NaN,NaN,cre,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20220420234500,https://www.cjvr.com/2022/04/20/first-nations-...
125667,1040383218,20220420,202204,2022,2022.3014,cre,CREE,NaN,NaN,cre,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20220420234500,https://www.cjvr.com/2022/04/20/first-nations-...
125668,1040383219,20220420,202204,2022,2022.3014,telOPP,TELUGU,NaN,NaN,tel,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20220420234500,https://www.deccanchronicle.com/nation/politic...


Each event in the data is highly denormalized for ease of querying and coded using a hierachical coding system called [CAMEO](http://data.gdeltproject.org/documentation/CAMEO.Manual.1.1b3.pdf) - Conflict and Mediation Event Observations

In [6]:
events[['EventCode', 'CAMEOCodeDescription']]

,EventCode,CAMEOCodeDescription
0,012,Make pessimistic comment
1,020,"Appeal, not specified below"
2,0213,Appeal for judicial cooperation
3,043,Host a visit
4,0311,Express intent to cooperate economically
...,...,...
125665,060,"Engage in material cooperation, not spec below"
125666,073,Provide humanitarian aid
125667,090,"Investigate, not specified below"
125668,043,Host a visit


We can query and filter event data directly using the Pandas dataframe

In [92]:
# Find all events that were geolocated in Ukraine that have a valid a Lat / Long
uka_events = events[(events.ActionGeo_CountryCode == 'UP')]
uka_events.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51845 entries, 79 to 125655
Data columns (total 62 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   GLOBALEVENTID          51845 non-null  int64  
 1   SQLDATE                51845 non-null  int64  
 2   MonthYear              51845 non-null  int64  
 3   Year                   51845 non-null  int64  
 4   FractionDate           51845 non-null  float64
 5   Actor1Code             49230 non-null  object 
 6   Actor1Name             49230 non-null  object 
 7   Actor1CountryCode      38586 non-null  object 
 8   Actor1KnownGroupCode   1402 non-null   object 
 9   Actor1EthnicCode       92 non-null     object 
 10  Actor1Religion1Code    675 non-null    object 
 11  Actor1Religion2Code    281 non-null    object 
 12  Actor1Type1Code        17962 non-null  object 
 13  Actor1Type2Code        1821 non-null   object 
 14  Actor1Type3Code        85 non-null     object 
 15  

In [81]:
uka_events['CAMEOCodeDescription']

79         Use conventional military force, not specifie...
83                             Consult, not specified below
87                                             Make a visit
119                            Consult, not specified below
121                            Consult, not specified below
                                ...                        
125624                               Cooperate economically
125625                         Consult, not specified below
125648                                    Praise or endorse
125651                                    Praise or endorse
125655                                         Make a visit
Name: CAMEOCodeDescription, Length: 51845, dtype: object

In [113]:
# Plot these events on a map
import folium
from folium.plugins import FastMarkerCluster
uka_map = folium.Map(
    location=[48., 31.], 
    tiles="Stamen Toner",
    zoom_start=6
)
uka_map

In [114]:
uka_events_sample = uka_events.sample(n=100)
for r in uka_events_sample.itertuples():
    m = folium.Marker(location=[r.ActionGeo_Lat, r.ActionGeo_Long],
                      icon=folium.Icon(color="red", icon="info-sign"),
                      tooltip=r.EventCode + ' ' + r.CAMEOCodeDescription
                     )
    m.add_to(uka_map)
uka_map